# Reducing pointing data

During the SUMMIT-4829 run, we obtained data at lower elevation to improve the existing pointing model. 

The data was taken in a slightly different way. Instead of centering the start in the CCD and storing the data for the pointing model we simply ran the wavefront analysis to collimate the optics, registered the position in the pointing and took an acquisition image, so we could later measure the offset and apply that to the registered position.

This notebooks is intended to gather the information about the data taken during a run and store it in a file that can be read and processed offline to generate pointing files.
These pointing files can then be used with `tpoint` to compute pointing models.

The next step will be to measure the position of the brightest start in the field, compute the offset with respect to the center of the field and add that to the data generated by the pointing. 
This will be done on a separate notebook.

## Parameterized notebook

This notebook is parameterized, which means it could be run with tools like Papermill as part of a data analysis pipeline.

## Requirements

In order to run this notebook you will need an updated version of `QuickFrameMeasurementTask`. 
By the time of this writting this library was still not integrated to the DM stack, so you may need to set it up manually. 

Assuming you are in one of the nublado environments (tts, nts, summit, etc), open a terminal and do the following:

```console
source ${LOADSTACK}
cd ${HOME}/WORK
git clone https://github.com/lsst-sitcom/rapid_analysis.git
cd rapid_analysis/
eups declare -r . -t $USER
```

Then, on your `${HOME}/notebooks/.user_setups` file, add the following like:

```
setup rapid_analysis -t $USER
```

In [ ]:
import os 
import pickle
import logging

import numpy as np

from astropy.time import Time
from datetime import timedelta, datetime

import lsst_efd_client

from lsst.pipe.base.struct import Struct
from lsst.pipe.tasks.quickFrameMeasurement import QuickFrameMeasurementTask

from lsst.rapid.analysis import BestEffortIsr

from lsst.ts.observing.utilities.auxtel.latiss.getters import get_image


In [ ]:
%matplotlib inline

In [ ]:
log = logging.getLogger("reducing_pointing_data")

## Notebook Parameters

The next cell define the notebook parameters

### Date of the observation

Basically the notebook take a year/month/day and time-window. It will then use that time spam to look for pointing data and associated images to analyse.

### Define timestamp when the data was taken

The next cell defines the dates when the data was taken, it is used by tthe following query to determine when to look for pointing component data registration

### SUMMIT-4829

```
start = Time('2021-02-18T00:00:00')
end = Time('2021-02-20T00:00:00')
```

### SUMMIT-5025

```
start = Time('2021-03-22T00:00:00')
end = Time('2021-03-25T00:00:00')
```

In [ ]:
year=2021
month=6
day=9
time_window=3
data_path = "/project/shared/auxTel/"
efd_name = "ldf_stable_efd"

In [ ]:
if efd_name not in lsst_efd_client.EfdClient.list_efd_names():
    raise RuntimeError(f"Unrecognizable efd_name ({efd_name}). Must be one of {lsst_efd_client.EfdClient.list_efd_names()}.")

In [ ]:
client = lsst_efd_client.EfdClient(efd_name)

In [ ]:
start = Time(f"{year}-{month:02d}-{day:02d}T00:00:00")
end = Time(f"{year}-{month:02d}-{day+time_window:02d}T00:00:00")

In [ ]:
log.debug(f"{start}, {end}")

# Data analysis

Look from when pointAddData command was sent to the pointing. These will mark the times when we registered the positions.

In [ ]:
timestamp = f"time >= '{start}+00:00' AND time <= '{end}+00:00'"
query = f'SELECT "expectedAzimuth", "expectedElevation", "measuredAzimuth", "measuredElevation", "measuredRotator" '\
        f'FROM "efd"."autogen"."lsst.sal.ATPtg.logevent_pointData" WHERE {timestamp}'

In [ ]:
log.debug(query)

In [ ]:
point_data = await client.influx_client.query(query)

In [ ]:
if len(point_data) == 0:
    raise RuntimeError(f"No pointing data in the specified time window: {start} - {end}")

### Measure ATAOS corrections offsets

The offsets with respect to the hexapod x/y corrections are mapped into az/el corrections later. Since this is an asynchronous event, we need to query for all instances in the time window we are analysing and then match each entry with the associated image/pointing data taken. 

In [ ]:
ataos_correction_offsets = await client.select_time_series(
    'lsst.sal.ATAOS.logevent_correctionOffsets', 
    [
        "x", 
        "y", 
        "z"
    ], 
    start.tai, 
    end.tai
)    

In [ ]:
log.debug(f"Found {len(ataos_correction_offsets)} ATAOS correctionOffsets")

## Finding acquisition images

Now we have the timestamps for when the telescope position was registered, we need to find the acquisition images.

The images where taken before registering the position so we need to look ~40s before the command was sent.

In [ ]:
acq_image_name = []
elevation = []
rotator_1 = []
rotator_2 = []
hexapod_x = []
hexapod_y = []
hexapod_z = []
ataos_correction_x = []
ataos_correction_y = []
ataos_correction_z = []


for time_reg in point_data.index:
    
    imageName = await client.select_time_series(
        'lsst.sal.ATCamera.logevent_endReadout',
        [
            "imageName"
        ], 
        Time(time_reg.to_julian_date(), format="jd", scale="tai") - timedelta(seconds=20),
        Time(time_reg.to_julian_date(), format="jd", scale="tai")
    )


    # It may happen that a image is not taken (or fails to be taken) when we register the position.
    # In this cases, add `None` to the acq_obsid list. These data will later be ignored.
    if hasattr(imageName, "imageName"):
        acq_image_name.append(imageName["imageName"][-1])
    else:
        acq_image_name.append(None)

    mount_Nasmyth_Encoders = await client.select_packed_time_series(
        "lsst.sal.ATMCS.mount_Nasmyth_Encoders",
        [
            "nasmyth1CalculatedAngle", 
            "nasmyth2CalculatedAngle"
        ],
        Time(time_reg).tai - timedelta(seconds=1.5), 
        Time(time_reg).tai + timedelta(seconds=1.5)
    )
    
    rotator_1.append(np.mean(mount_Nasmyth_Encoders["nasmyth1CalculatedAngle"]))
    rotator_2.append(np.mean(mount_Nasmyth_Encoders["nasmyth2CalculatedAngle"]))
    
    elevationCalculatedAngle = await client.select_packed_time_series(
        "lsst.sal.ATMCS.mount_AzEl_Encoders",
        [
            "elevationCalculatedAngle"
        ],
        Time(time_reg).tai - timedelta(seconds=1.5),
        Time(time_reg).tai + timedelta(seconds=1.5)
    )

    elevation.append(np.mean(elevationCalculatedAngle["elevationCalculatedAngle"]))
    
    hexapod_positions = await client.select_time_series(
        'lsst.sal.ATHexapod.positionStatus',
        [
            "reportedPosition0",
            "reportedPosition1",
            "reportedPosition2"
        ],
        Time(time_reg).tai - timedelta(seconds=1.5), 
        Time(time_reg).tai + timedelta(seconds=1.5)
    )

    
    hexapod_x.append(np.mean(hexapod_positions["reportedPosition0"]))
    hexapod_y.append(np.mean(hexapod_positions["reportedPosition1"]))
    hexapod_z.append(np.mean(hexapod_positions["reportedPosition2"]))
    
    indx = ataos_correction_offsets.index.get_loc(time_reg, method='nearest')

    x, y, z = (
        (
            ataos_correction_offsets["x"][indx],
            ataos_correction_offsets["y"][indx],
            ataos_correction_offsets["z"][indx],
        )
        if time_reg > ataos_correction_offsets.index[indx]
        else (
            ataos_correction_offsets["x"][indx - 1],
            ataos_correction_offsets["y"][indx - 1],
            ataos_correction_offsets["z"][indx - 1],
        )
    )

    ataos_correction_x.append(x)
    ataos_correction_y.append(y)
    ataos_correction_z.append(z)

In [ ]:
log.info("EFD mining completed...")

## Measuring star position in each image

The following cells use the `QuickFrameMeasurementTask` to compute the position of the brightest source in the field. The data is stored in a structure that is later augmented with additional information (mined from the EFD) that is required to compute the pointing files.

The main idea behind processing the data and storing the results in a file is because the process bellow can take quite a while to finish.

In [ ]:
qm_config = QuickFrameMeasurementTask.ConfigClass()

In [ ]:
qm = QuickFrameMeasurementTask(config=qm_config)

In [ ]:
best_effort_isr = BestEffortIsr(data_path)

In [ ]:
log.info(f"Running QuickFrameMeasurementTask in {len(acq_image_name)} images. This may take some time.")
log.debug(f"Data path: {data_path}")

In [ ]:
brightest_source_centroid = []

for image_name in acq_image_name:
    if image_name is None:
        result = Struct()
        brightest_source_centroid.append(result)
        continue
    _, _, day_obs, seq_num = image_name.split("_")
    day_obs = f"{day_obs[0:4]}-{day_obs[4:6]}-{day_obs[6:8]}"
    exp = await get_image(
        dict(dayObs=day_obs, seqNum=int(seq_num)),
        best_effort_isr,
        timeout=10,
    )
    result = qm.run(exp)

    brightest_source_centroid.append(result)

In [ ]:
log.info("QuickFrameMeasurementTask Done")

## Pre-compute data

Each entry in `brightest_source_centroid` contains the centroid of brightest source in the field in pixel coordinates.

The data is augmented with the information queried from the EFD; telescope position, hexapod position, AOS offsets and pointing data.
These are needed to compute the corrected pointing data information.


In [ ]:
angle = np.array(elevation) - np.array(rotator_2) + 90.0

In [ ]:
for indx in range(len(brightest_source_centroid)):
    brightest_source_centroid[indx].image_name = acq_image_name[indx]
    brightest_source_centroid[indx].elevation = elevation[indx]
    brightest_source_centroid[indx].rotator_2 = rotator_2[indx]    
    brightest_source_centroid[indx].angle = angle[indx]
    brightest_source_centroid[indx].point_data = dict([(key, point_data[key][indx]) for key in ("expectedAzimuth", "expectedElevation", "measuredAzimuth", "measuredElevation", "measuredRotator")])
    brightest_source_centroid[indx].hexapod_x = hexapod_x[indx]
    brightest_source_centroid[indx].hexapod_y = hexapod_y[indx]
    brightest_source_centroid[indx].hexapod_z = hexapod_z[indx]
    brightest_source_centroid[indx].aos_offset = dict(x=ataos_correction_x[indx], y=ataos_correction_y[indx], z=ataos_correction_z[indx])

## Store computed data into a pickle file

The data is now stored in a pickle file that can later be read and processed as needed.

In [ ]:
out_pointing_data_path = f"data/{year}{month:02d}{day:02d}"
out_pointing_data_name = f"AT_point_data_{year}{month:02d}{day:02d}_tw{time_window:03d}.pickle"

In [ ]:
if not os.path.exists(out_pointing_data_path):
    log.debug(f"Output destination ({out_pointing_data_path}) does not exists, creating directory tree.")
    os.makedirs(out_pointing_data_path)

In [ ]:
out_pointing_data_file = os.path.join(out_pointing_data_path, out_pointing_data_name)

In [ ]:
log.info(f"Writting data to {out_pointing_data_file}")

In [ ]:
with open(out_pointing_data_file, "wb") as fp:
    pickle.dump(brightest_source_centroid, fp)

## End

The file is now ready to be analysed with tpoint to produce a new pointing model.